In [16]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier

from ipywidgets import Image
from io import StringIO
import pydotplus
from sklearn.tree import export_graphviz

import math as m
import collections as coll

In [17]:
# задамо графіки у форматі .svg, щоби вони мали кращу чіткість
%config InlineBackend.figure_format = 'svg' 

# підвищимо розмір графіків за замовчуванням
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 7, 5

In [18]:
# Створення датафрейму з dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

# Деякі значення ознак є в тестовій підвибірці, але немає в навчальній і навпаки
def intersect_features(train, test):
    common_feat = list( set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]


In [19]:

features = ['Зовнішність', 'Алкоголь_в_напої',
            'Рівень_харизми', 'Витрачені_гроші']


In [20]:

df_train = {}
df_train['Зовнішність'] = ['приємна', 'приємна', 'приємна', 'відштовхуюча',
                         'відштовхуюча', 'відштовхуюча', 'приємна'] 
df_train['Алкоголь_в_напої'] = ['так', 'так', 'ні', 'ні', 'так', 'так', 'так']
df_train['Рівень_харизми'] = ['високий', 'низький', 'середній', 'середній', 'низький',
                                   'високий', 'середній']
df_train['Витрачені_гроші'] = ['багато', 'мало', 'багато', 'мало', 'багато',
                                  'багато', 'багато']
df_train['Поїде'] = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])

df_train = create_df(df_train, features)
df_train

,Поїде,Зовнішність_відштовхуюча,Зовнішність_приємна,Алкоголь_в_напої_ні,Алкоголь_в_напої_так,Рівень_харизми_високий,Рівень_харизми_низький,Рівень_харизми_середній,Витрачені_гроші_багато,Витрачені_гроші_мало
0,0,0,1,0,1,1,0,0,1,0
1,1,0,1,0,1,0,1,0,0,1
2,0,0,1,1,0,0,0,1,1,0
3,1,1,0,1,0,0,0,1,0,1
4,1,1,0,0,1,0,1,0,1,0
5,0,1,0,0,1,1,0,0,1,0
6,0,0,1,0,1,0,0,1,1,0


In [21]:
willride = df_train[(df_train['Поїде'] == 1) & (df_train['Зовнішність_приємна'] == 0)]['Поїде'].count()
cases = df_train[df_train['Зовнішність_приємна'] == 0]['Поїде'].count()
wontride = cases - willride

S2 = -(willride / cases) * m.log((willride / cases), 2) - (wontride / cases) * m.log((wontride / cases), 2)
S2

0.9182958340544896

In [22]:
def entropy(a_list):
    
    numOfElements = len(a_list)
    elements = coll.Counter([el for el in a_list])

    entropy = 0

    for elm in elements:
      ni = elements[elm]
      pi = ni / numOfElements
      entropy += pi * (m.log(pi, 2))

    return entropy * -1

balls = [1 for i in range(9)] + [0 for i in range(11)]

balls_left  = [1 for i in range(8)] + [0 for i in range(5)] # 8 синіх і 5 жовтих
balls_right = [1 for i in range(1)] + [0 for i in range(6)] # 1 синя і 6 жовтих

print(entropy(balls)) # 9 синіх і 11 жовтих

0.9927744539878084


In [23]:
def information_gain(root, left, right):
  N = len(root)
  Ni1 = len(left)
  Ni2 = len(right)

  return entropy(root) - Ni1 / N * entropy(left) - Ni2 / N * entropy(right)

information_gain(balls, balls_left, balls_right)

0.16088518841412436

In [24]:
data_train_url = 'https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-03_classification/adult_train.csv'
data_train = pd.read_csv(data_train_url, sep=';')

data_train.tail()



tree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=17)

tree.fit(X_train, Y_train.values)

dot_data = StringIO()
export_graphviz(tree, feature_names=X_train.columns, 
                out_file=dot_data, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(value=graph.create_png())



Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\xd7\x00\x00\x01\xf1\x08\x02\x00\x00\x00\x00\xca\…

In [11]:
data_test_url = 'https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-03_classification/adult_test.csv'
data_test = pd.read_csv(data_test_url, sep=';')



data_test.tail()

,Age,Workclass,fnlwgt,Education,Education_Num,Martial_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Country,Target
16277,39,Private,215419.0,Bachelors,13.0,Divorced,Prof-specialty,Not-in-family,White,Female,0.0,0.0,36.0,United-States,<=50K.
16278,64,NaN,321403.0,HS-grad,9.0,Widowed,NaN,Other-relative,Black,Male,0.0,0.0,40.0,United-States,<=50K.
16279,38,Private,374983.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
16280,44,Private,83891.0,Bachelors,13.0,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455.0,0.0,40.0,United-States,<=50K.
16281,35,Self-emp-inc,182148.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,60.0,United-States,>50K.


In [13]:
data_test = data_test[(data_test['Target'] == ' >50K.') 
                      | (data_test['Target']==' <=50K.')]
# перекодуємо target в числове поле
data_train.loc[data_train['Target'] == ' <=50K', 'Target'] = 0
data_train.loc[data_train['Target'] == ' >50K', 'Target'] = 1

data_test.loc[data_test['Target'] == ' <=50K.', 'Target'] = 0
data_test.loc[data_test['Target'] == ' >50K.', 'Target'] = 1


In [14]:
data_test['Age'] = data_test['Age'].astype(int)
data_test['fnlwgt'] = data_test['fnlwgt'].astype(int)
data_test['Education_Num'] = data_test['Education_Num'].astype(int)
data_test['Capital_Gain'] = data_test['Capital_Gain'].astype(int)
data_test['Capital_Loss'] = data_test['Capital_Loss'].astype(int)
data_test['Hours_per_week'] = data_test['Hours_per_week'].astype(int)

categorical_columns_train = [c for c in data_train.columns 
                             if data_train[c].dtype.name == 'object']
numerical_columns_train = [c for c in data_train.columns 
                           if data_train[c].dtype.name != 'object']

categorical_columns_test = [c for c in data_test.columns 
                            if data_test[c].dtype.name == 'object']
numerical_columns_test = [c for c in data_test.columns 
                          if data_test[c].dtype.name != 'object']

for c in categorical_columns_train:
    data_train[c] = data_train[c].fillna(data_train[c].mode())
for c in categorical_columns_test:
    data_test[c] = data_test[c].fillna(data_train[c].mode())
    
for c in numerical_columns_train:
    data_train[c] = data_train[c].fillna(data_train[c].median())
for c in numerical_columns_test:
    data_test[c] = data_test[c].fillna(data_train[c].median())

data_train = pd.concat([data_train, pd.get_dummies(data_train['Workclass'], 
                                                   prefix="Workclass"),
                      pd.get_dummies(data_train['Education'], prefix="Education"),
                      pd.get_dummies(data_train['Martial_Status'], prefix="Martial_Status"),
                      pd.get_dummies(data_train['Occupation'], prefix="Occupation"),
                      pd.get_dummies(data_train['Relationship'], prefix="Relationship"),
                      pd.get_dummies(data_train['Race'], prefix="Race"),
                      pd.get_dummies(data_train['Sex'], prefix="Sex"),
                      pd.get_dummies(data_train['Country'], prefix="Country")],
                     axis=1)

data_test = pd.concat([data_test, pd.get_dummies(data_test['Workclass'], prefix="Workclass"),
                      pd.get_dummies(data_test['Education'], prefix="Education"),
                      pd.get_dummies(data_test['Martial_Status'], prefix="Martial_Status"),
                      pd.get_dummies(data_test['Occupation'], prefix="Occupation"),
                      pd.get_dummies(data_test['Relationship'], prefix="Relationship"),
                      pd.get_dummies(data_test['Race'], prefix="Race"),
                      pd.get_dummies(data_test['Sex'], prefix="Sex"),
                      pd.get_dummies(data_test['Country'], prefix="Country")],
                     axis=1)

data_train.drop(['Workclass', 'Education', 'Martial_Status',
                 'Occupation', 'Relationship', 'Race', 'Sex', 'Country'],
                axis=1, inplace=True)
data_test.drop(['Workclass', 'Education', 'Martial_Status', 'Occupation', 
                'Relationship', 'Race', 'Sex', 'Country'],
               axis=1, inplace=True)

data_test['Country_ Holand-Netherlands'] = np.zeros([data_test.shape[0], 1])

X_train=data_train.drop(['Target'], axis=1)
Y_train = data_train['Target']

X_test=data_test.drop(['Target'], axis=1)
Y_test = data_test['Target']

In [25]:

tree = DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=16)

tree.fit(X_train, Y_train.values)

dot_data = StringIO()
export_graphviz(tree, feature_names=X_train.columns, 
                out_file=dot_data, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(value=graph.create_png())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x034\x00\x00\x01g\x08\x06\x00\x00\x00p\xf7\x0c\x07\x…

In [26]:
treePredictions = tree.predict(X_test)

right = 0
i = 0

for v in Y_test:
  if v == treePredictions[i]:
    right += 1
  i += 1

right * 100 / len(treePredictions)

83.06615072784227

In [27]:
newTree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=17)

newTree.fit(X_train, Y_train.values)
newTreePredictions = newTree.predict(X_test)

right = 0
i = 0

for v in Y_test:
  if v == newTreePredictions[i]:
    right += 1
  i += 1

right * 100 / len(newTreePredictions)

84.47884036607088

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

tree_params = {'criterion':['entropy'], 'max_depth':[2, 3, 4, 5, 6, 7, 8, 9, 10],  'random_state': [17]}

locally_best_tree = GridSearchCV(DecisionTreeClassifier(), tree_params, cv = 5)                  

locally_best_tree.fit(X_train, Y_train.values)

print("Best params:", locally_best_tree.best_params_)
print("Best cross validaton score", locally_best_tree.best_score_)

Best params: {'criterion': 'entropy', 'max_depth': 8, 'random_state': 17}
Best cross validaton score 0.8551643217062379


In [29]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=8, random_state=17)

tree.fit(X_train, Y_train.values)

treePredictions = tree.predict(X_train)

right = 0
i = 0

for v in Y_train:
  if v == treePredictions[i]:
    right += 1
  i += 1

right * 100 / len(treePredictions)

85.98323147323485

In [30]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=16)

tree.fit(X_train, Y_train.values)

treePredictions = tree.predict(X_train)

right = 0
i = 0

for v in Y_train:
  if v == treePredictions[i]:
    right += 1
  i += 1

right * 100 / len(treePredictions)

82.82300912134149